### Initialization

In [55]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from tqdm import tqdm

import ipywidgets as widgets
from IPython.display import display

In [56]:
json_file = open("API_key.json") # load the API key
API_KEY = json.load(json_file)["API_key"]
json_file.close()

watcher = LolWatcher(API_KEY) # initialization of Lol watcher to retrieve data

### Seconds to hours, minutes and seconds

In [57]:
def sec_to_hms(seconds):
    
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%02d minutes %02d seconds" % (minutes, seconds)

### Get puuid of a user

In [58]:
def get_puuid(summoner_name, region='euw1'):
    
    puuid = watcher.summoner.by_name(region, summoner_name)["puuid"]
    
    return puuid

### Get item name from ID

In [59]:
def get_item_name(item_id):
    
    item_version = watcher.data_dragon.versions_for_region('euw1')['n']['item']
    item_name = watcher.data_dragon.items(item_version)['data'][str(item_id)]['name']
    
    return item_name

### Add data for a specific champion from the history of a user

Allows you to add the data of a specific player to the database. 

You have to give the champion whose data you want to retrieve, for example: "Sett" and then the name of the player for whom you want to retrieve the data of the champion "Sett". This way you can retrieve the items that a player has bought while playing Sett and the enemies that he has faced in the last n games.

In [60]:
def add_data(Summoner_name,target_champ,num_matches=50,region=['euw1','europe']):

    # Summoner_name : Name of the player whose data you want to add, it can be a list
    #
    # target_champ : The champion whose data you want to add
    #
    # num_matches : Number of games to analyze starting from the last game (if the target champion does 
    # not appear in the last n matches, no data will be added)
    #
    # region : Europe = 'euw1','europe'
    #          Korea = 'kr','asia'
    
    for name in Summoner_name:
    
        player_puuid = get_puuid(name,region[0]) 
        my_matches = watcher.match.matchlist_by_puuid(region[1],player_puuid,count=num_matches)

        start_time = time.time()
        date = datetime.today().strftime('%d/%m/%Y')

        row = []
        champ_list = []
        item_list = []

        df = pd.read_csv('Lol_data.csv')

        print("If the loading stops, it means that the API has reached the maximum number of requests.\n"+
              "You have to wait 2 minute before the loading resumes.\n"+
              "\nAdding "+target_champ+" data of the player "+name+" :\n")

        for L in tqdm(range(num_matches)):
            for i in range(10): 
                try:
                    test_match = watcher.match.by_id(region[1], my_matches[L])
                except:
                    pass
                champion = test_match['info']['participants'][i]['championName']
                champ_list = np.append(champ_list,champion) # get the list of champions in the game

            patch = test_match['info']['gameVersion'].split(".")
            patch_str = patch[0]+'.'+patch[1]

            if target_champ not in champ_list: # check if target champion is in the game
                champ_list = []
                row = []
                continue

            target_champ_idx = np.where(champ_list == target_champ)[0][0]

            if target_champ_idx < 5: # get only the enemies 
                enemy_list = champ_list[5:10]
            else:
                enemy_list = champ_list[0:5]

            row = np.insert(enemy_list,0,target_champ)

            for k in range(6): # get the items of the target champion
                try:
                    item_n_str = 'item'+str(k)
                    item = test_match['info']['participants'][target_champ_idx][item_n_str]
                    row = np.append(row,item) 
                except Exception as error:
                    row = np.append(row,'no item')

            row = np.append(row,[player_puuid,my_matches[L],date,patch_str])

            if len(row):
                df_len = len(df)
                df.loc[df_len] = row

            champ_list = []
            row = []

        df.to_csv('Lol_data.csv',index=False) # adding data to csv, encoding cp1252 for specific characters
        end_time = time.time()-start_time
        print("\nData added in "+sec_to_hms(end_time)+'\n')

In [61]:
summoners_history_euw = ['Nortu','Gudule','Akn Hanima','TokÌta Ohma','Bechamelle','MENTALLY 1MMUNE','LFT BRK TR','Ping An Xi Lè',
                         'EG PEIWAN Sao','HMPeiWan Yorkie','BOI NEXT DOOR','I am N1Ty','amongus potion12','no i do not care',
                         'Zzzzzbr','Doope','sokrfd','Flynn 7','dopa farter','HMPeiWan NanBao','StDaimoniac','Chen Noll','TwTv NMB Jordan',
                         'Frustrated Kiddo','littlehigh01']

summoners_history_kr = ['아 가 리','tenten12qq','이걸 세트가','ren sheng ru xi','칸야블','첼까지닉변안함','전승트','하이눈 마이','카시스 페델리안',
                        '세트말고할거없음','예전에 내가 아냐','평화로운 친구','kamuE','옐로우핀','세트의 꿈','겜좀집중해서하자','남장구',
                        '문신 덩크충','JYPBoss','Lestiny','물냉파','WWWave','동물특공대 미 셸','God Thanatos','WVVWVWVVWVWWVW']

summoners = []
target_champ = 'Sett'
num_matches = 50
korea = ['kr','asia']
europe = ['euw1','europe']

# add_data(summoners,target_champ,num_matches,europe)

### Web app to add data to database

In [68]:
get_name_box = widgets.Text(placeholder='Enter a summoner name',description='Username : ')
get_region_box = widgets.Dropdown(options=['Europe west','Korea'],description='Region : ')
get_targetchamp_box = widgets.Text(placeholder='Enter the name of a champion',description='Champion : ')
get_nummatches_box = widgets.IntSlider(value=50,min=1,max=100,step=1,description='Matches')
adddata_button = widgets.Button(description='Add data',button_style='success',tooltip='Click to add data',icon='database')

display(get_name_box,get_region_box,get_targetchamp_box,get_nummatches_box,adddata_button)

def on_button_clicked(b):
    
    summoner = get_name_box.value
    target_champ = get_targetchamp_box.value
    num_matches = get_nummatches_box.value
    region_choice = get_region_box.value
    
    if region_choice == 'Europe west':
        region = ['euw1','europe']
    elif region_choice == 'Korea':
        region = ['kr','asia']
    
    add_data([summoner],target_champ,num_matches,region)
    
adddata_button.on_click(on_button_clicked)

Text(value='', description='Username : ', placeholder='Enter a summoner name')

Dropdown(description='Region : ', options=('Europe west', 'Korea'), value='Europe west')

Text(value='', description='Champion : ', placeholder='Enter the name of a champion')

IntSlider(value=50, description='Matches', min=1)

Button(button_style='success', description='Add data', icon='database', style=ButtonStyle(), tooltip='Click to…